In [15]:
import pandas as pd
import requests
import pytest
import datetime as dt
import json
import sys
import os
import sqlite3 as db

In [16]:
ebird_token = 'j6c7l80ga2ib'
db_name = 'trailheadDirectBirds_sous.db'

In [17]:
##connect to database
def connectDB():
    try:
        cnx = db.connect(db_name)
    except Exception as cnxError:
        raise UserWarning(f'Unable to connect to database due to: {cnxError}')
    return cnx

In [88]:
def get_hotspots():
    from datetime import timedelta
    trailheadHotspots = pd.DataFrame()
    histHotObs = pd.DataFrame()
    cnx = connectDB()
    try:
        #get the latest hotspots from the database
        #trailheadHotspots = pd.read_sql('select * from Hotspots',con=cnx,parse_dates=['latestObsDt','latestUpdate'])   #Excluding Discovery Park until a later date
        trailheadHotspots = pd.read_sql('select * from Hotspots where Hotspots.StopName not like "DiscoveryPark%"',con=cnx,parse_dates=['latestObsDt','latestUpdate'])
        trailheadHotspots.set_index('index',inplace=True)
        trailheadHotspots.sort_values(by=['locId','latestObsDt'],ascending=False,ignore_index=True,inplace=True)
        uniqueHotspots = trailheadHotspots[['locId','latestObsDt']].drop_duplicates(subset=['locId'],keep='first')
        #get hotspots from the observations in the database
        histHotObs = pd.read_sql('select locId,obsDt,loadDate from historicObservations',con=cnx,parse_dates=['obsDt','loadDate'])
        min_obsDt = min(histHotObs['obsDt'])
        histHotObs.sort_values(by=['locId','obsDt'],ascending=False,ignore_index=True,inplace=True)
        histHotObs.drop_duplicates(subset=['locId'],keep='first',inplace=True)
        #merge the tables
        hotspots = pd.merge(left=uniqueHotspots,right=histHotObs,how='left',left_on='locId',right_on='locId')
        #remove hotspots without a recent checklist
        agingMask = hotspots.apply(lambda y: bool(y.loadDate is not(pd.NaT) or bool(y.latestObsDt < y.loadDate)),axis=1)
        hotspots = hotspots[~agingMask]
        #replace null, NaT observation date values with the earliest value found in the table
        hotspots.loc[(hotspots['obsDt'].isna()) | (hotspots['obsDt'].isnull()) | (hotspots['obsDt'] == ''),'obsDt'] = min_obsDt
        #recency check
        for x in hotspots:
            dateDiff = hotspots['latestObsDt']-hotspots['obsDt']
            hotspots['dateDiff'] = dateDiff.dt.days
        hotspots['run'] = hotspots['dateDiff'].between(7,700,inclusive='both')
        hotspots = hotspots[hotspots['run'] == True]
        dfk = hotspots[['locId','latestObsDt','obsDt']]
    except Exception as ee:
        raise UserWarning(ee)
    finally: cnx.close()
    return dfk

In [89]:
gg = get_hotspots()
gg

,locId,latestObsDt,obsDt
4,L7672326,2021-08-21 12:30:00,2021-05-31 15:27:00
10,L4703260,2022-02-12 16:08:00,2020-12-24 15:49:00
11,L450305,2022-01-28 11:00:00,2020-12-28 10:14:00
12,L4381196,2021-08-10 19:11:00,2020-04-28 12:18:00
13,L424117,2022-02-11 05:27:00,2020-12-30 05:13:00
14,L3943215,2022-02-06 12:51:00,2020-08-13 12:19:00
16,L3352189,2021-11-30 10:12:00,2020-01-06 14:15:00
20,L1167737,2022-02-12 09:11:00,2020-12-04 10:09:00
21,L11570424,2021-08-31 15:56:00,2020-08-12 17:14:00
24,L1034830,2021-06-16 12:27:00,2019-12-22 11:11:00


##Checking for new hotspots and observations
1. Compare the latest trailheadHotspots eBird locId's with the eBird locId's in the cooking DB
2. For new locId's:
    1. Fetch historical data from the beginning of 2019 to today()-1 from the eBird REST API
    2. Concat and export the results as a .csv
3. For locId's with observations in the DB, obtain the most recent observation date from the eBird REST API for that hotspot:
    1. Make a list of locId's WHERE eBird's latest observation date is more recent than the latest observation date in the DB. Using the most recent DB obsDt as a start date and the most recent eBird API obsDt as an end date, fetch historical sightings from the eBird service
    2. IF the latest obsDt for a locId/hotspot from eBird is within 15 days of the latest obsDt in the DB, remove the locId from the list of locId's to be passed into the historical eBird sighting service
        - TODO: #20 write 'aging' and 'decayed' testing, error logging

In [33]:
##TODO #22 recency refactor: get the latest date some other way; eBird hotspot info does not contain the latest date
##TODO #23 recency refactor: pare down results of hotspot batches to match known Trailhead Direct eBird hotspots

In [45]:
def get_histObs():
    import time
    from datetime import timedelta
    ObsHist = []
    cnx = connectDB()
    try:
        dfk_test = pd.DataFrame(get_hotspots().head(2))
        if dfk_test.empty:
            raise Exception('The provided dataframe is empty. There are either no eBird hotspots with new checklists or an error has occurred in the evaluation of the get_hotposts() function.')
        for locId in dfk_test.itertuples():
            startDate = locId.obsDt + timedelta(days=1)
            for x in pd.date_range(start=startDate,end=locId.latestObsDt,freq='D'):
                time.sleep(0.5)
                ymd = '{}/{}/{}'.format(x.year,x.month,x.day)
                ebird_baseUrl = 'https://api.ebird.org/v2/data/obs/'
                ebird_url = ebird_baseUrl + locId.locId + '/historic/' + ymd
                ebird_auth = {'X-eBirdApiToken': ebird_token}
                ebird_params = {
                    'fmt':'json',
                    'detail':'simple'
                }
                ebird_request = requests.get(ebird_url,params=ebird_params,headers=ebird_auth)
                if ebird_request.status_code == requests.codes.ok:
                    ebird_resp = pd.DataFrame(ebird_request.json())
                    ebird_resp['loadDate'] = dt.datetime.today()
                    ObsHist.append(ebird_resp)
                ebird_request.raise_for_status()
        historicObservations = pd.concat(ObsHist)
        historicObservations.to_sql(name='historicObservations_cooking',con=cnx,if_exists='append')
        cnx.close()
    except Exception as exc:
        raise exc
    return historicObservations

In [46]:
xr = get_histObs()
xr.head()

,loadDate,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
0,2022-02-23 12:20:11.286687,grnher,Green Heron,Butorides virescens,L4920968,Pickering Place Pond,2021-01-01 11:53,1.0,47.54811,-122.052462,True,False,False,S97993349
0,2022-02-23 12:20:14.925683,buffle,Bufflehead,Bucephala albeola,L4920968,Pickering Place Pond,2021-01-05 16:14,1.0,47.54811,-122.052462,True,False,False,S78749960
1,2022-02-23 12:20:14.925683,hoomer,Hooded Merganser,Lophodytes cucullatus,L4920968,Pickering Place Pond,2021-01-05 16:14,1.0,47.54811,-122.052462,True,False,False,S78749960
2,2022-02-23 12:20:14.925683,gockin,Golden-crowned Kinglet,Regulus satrapa,L4920968,Pickering Place Pond,2021-01-05 16:14,1.0,47.54811,-122.052462,True,False,False,S78749960
3,2022-02-23 12:20:14.925683,sonspa,Song Sparrow,Melospiza melodia,L4920968,Pickering Place Pond,2021-01-05 16:14,1.0,47.54811,-122.052462,True,False,False,S78749960


In [38]:
ObsHist = []
import time

for locId in dfk_test.itertuples():
    for x in pd.date_range(start=locId.obsDt,end=locId.latestObsDt,freq='D'):
        time.sleep(0.3)
        ymd = '{}/{}/{}'.format(x.year,x.month,x.day)
        ebird_baseUrl = 'https://api.ebird.org/v2/data/obs/'
        ebird_url = ebird_baseUrl + locId.locId + '/historic/' + ymd
        ebird_auth = {'X-eBirdApiToken': ebird_token}
        ebird_params = {
            'fmt':'json',
            'detail':'simple'
        }
        ebird_request = requests.get(ebird_url,params=ebird_params,headers=ebird_auth)
        if ebird_request.status_code == requests.codes.ok:
            ebird_resp = pd.DataFrame(ebird_request.json())
            ObsHist.append(ebird_resp)
        ebird_request.raise_for_status()

In [39]:
##TODO #29 write the new observations to the database --DONE
historicObservations = pd.concat(ObsHist)

In [40]:
historicObservations.to_sql(name='historicObservations_cooking',con=cnx,if_exists='append')
##close the connection
cnx.close()